In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import mt4_hst
%matplotlib inline

pd.options.display.max_rows = 4000

In [2]:
ticker = 'FPT'
df = mt4_hst.read_hst("../stock_datasets/" + ticker + "1440.hst")
df = df.sort_values('time')
df.sort_values(by='time', inplace=True)

In [3]:
TechnicalIndicator = ta.Strategy(
    name="Standard Technical Indicator in various research of automatic stock trading",
    description="SMA MACD RSI CCI ADX Bollinger",
    ta=[
        {"kind": "sma", "length": 10},
        {"kind": "rsi", "length": 14},
        {"kind": "cci", "length": 14},
        {"kind": "adx", "length": 14},
        {"kind": "bbands"},
        {"kind": "macd"},
    ]
)

In [7]:
df.ta.strategy(TechnicalIndicator)
df.dropna(inplace=True)

In [15]:
indicators = "close SMA_10 RSI_14 CCI_14_0.015 BBL_5_2.0 BBU_5_2.0 MACD_12_26_9".split()
df[indicators].head()

,close,SMA_10,RSI_14,CCI_14_0.015,BBL_5_2.0,BBU_5_2.0,MACD_12_26_9
33,25.202848,23.262031,74.643164,153.627647,21.926752,25.383161,1.620369
34,25.599743,23.639081,75.753119,166.128525,22.020349,26.321492,1.717492
35,24.607506,23.912939,67.766704,109.471672,22.667903,26.308971,1.694860
36,25.004400,24.119324,69.166974,96.690511,23.802747,25.967917,1.689475
37,23.774025,24.115355,60.406317,23.063643,23.596342,26.079067,1.567852


In [ ]:
df.ta.rsi(length=20, append=True)
df.ta.natr(length=20, scalar=1, append=True)
df.ta.log_return(length=5, append=True)
df.ta.log_return(length=20, append=True)
df.ta.percent_return(length=5, append=True)
df.ta.percent_return(length=20, append=True)

# trend setup
# df.ta.sma(50, append=True)
df['close>sma50'] = (df['close'] > df.ta.sma(50)).astype(int)
df['close>sma100'] = (df['close'] > df.ta.sma(100)).astype(int)
df['close>sma200'] = (df['close'] > df.ta.sma(200)).astype(int)

# df.ta.ema(10, append=True)
# df.ta.ema(20, append=True)
df['ema10>ema20'] = (df.ta.ema(10) > df.ta.ema(20)).astype(int)

donchian_20 = ta.donchian(df['high'], df['close'], lower_length=20, upper_length=20)
donchian_50 = ta.donchian(df['high'], df['close'], lower_length=50, upper_length=50)
df['higher_low'] = (donchian_20['DCL_20_20'] > donchian_50['DCL_50_50']).astype(int)
df['breakout'] = (df['close'] > donchian_20['DCU_20_20'].shift(1)).astype(int)

# volume confirm
df['volume_breakout'] = (df['volume'] > ta.sma(df['volume'], 20)).astype(int)

# df.ta.tsignals(ta.ema(df['close'], 10) > ta.ema(df['close'], 20), append=True)
# df.ta.log_return(append=True)
# df.ta.percent_return(append=True)
# df.ta.roc(append=True)
# df.ta.stoch(append=True)
# df.ta.stochrsi(append=True)
# df.ta.tsi(append=True)

# df.describe()
df.dropna(inplace=True)
df.head()
df.to_csv('temp/features.csv', index=False)